In [ ]:
# 출처 : https://wikidocs.net/159468
# 모델 불러오기
!pip install konlpy
!pip install -U sentence-transformers
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 테스트용 문구
doc = """
제가 (중고) 신입으로 취업하기 위해서 넘어야 할 벽이 몇가지 있었는데, 그 중의 하나가 코딩테스트입니다.
개인적으로 코딩테스트를 보지 않는 기업을 선호했으나, 최근 추세가 대부분의 회사는 코딩테스트가 필수적이더군요.
그래서 인터넷에서 검색해본 공부방법을 따라해봤으나, 저하고는 그다지 맞지 않는다라는 느낌을 받았습니다.
분명히 모든 일에는 효율적인 방향이 존재할 것이고, 인터넷에 올라와 있는 대부분의 방법은 효율적이지 않았습니다.

코딩테스트가 핫해져서일까요? 최근 코딩테스트 관련 서적 두 종이 비슷한 시기에 파이썬이라는 언어를 가지고 출판되었습니다.
그래서 초반에 구매를 선뜻 할 수는 없었습니다. 언제 파이썬 배워서 써먹지? 라는 생각이 먼저 들었습니다.
며칠 고민을 하다가, 파이썬이면 그래도 범용적으로 많이 쓰이는 언어니까 배워두면 나쁘지 않을 것 같다는 생각을 가지고 구매를 했습니다.
"""

In [ ]:
# 품사 태깅 및 명사 추출
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('제', 'Noun'), ('가', 'Josa'), ('(', 'Punctuation'), ('중고', 'Noun'), (')', 'Punctuation'), ('신입', 'Noun'), ('으로', 'Josa'), ('취업', 'Noun'), ('하기', 'Verb')]
명사 추출 : 제 중고 신입 취업 위해 벽 가지 그 중 하나 코딩 테스트 개인 코딩 테스트 보지 기업 선호 최근 추세 대부분 회사 코딩 테스트 필수 인터넷 검색 공부 방법 저 그다지 느낌 모든 일 효율 방향 존재 것 인터넷 대부분 방법 효율 코딩 테스트 핫 요 최근 코딩 테스트 관련 서적 두 종이 시기 파이썬 언어 가지 출판 초반 구매 선뜻 수 언제 파이썬 생각 먼저 며칠 고민 파이썬 범용 언어 것 생각 가지 구매


In [ ]:
# 단어 5개만 테스트 출력
n_gram_range = (1, 1)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('단어 개수 :',len(candidates))
print('단어 다섯개만 출력 :',candidates[:5])

단어 개수 : 43
단어 다섯개만 출력 : ['가지' '개인' '검색' '고민' '공부']


In [ ]:
# 다국어 sbert 호출
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
# 빈출 단어 5개 출력
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['취업', '효율', '인터넷', '공부', '코딩']


In [ ]:
# mmr 알고리즘(다양한 종류의 단어 호출) 함수 생성
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
# mmr 함수 실행
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['코딩', '인터넷', '공부', '효율', '기업']